In [1]:
import os

In [2]:
%pwd

'f:\\Files\\DS&ML\\E2E-Credit-Fraud-Detection\\Exp'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\Files\\DS&ML\\E2E-Credit-Fraud-Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    zip_file: Path
    unzip_file: Path

In [7]:
from project.utils.common import *
from project.constants import *

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_PATH,
        params_filepath = PARAMS_PATH,
        schema_filepath = SCHEMA_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            zip_file=config.zip_file,
            unzip_file=config.unzip_file 
        )

        return data_ingestion_config

In [9]:
import urllib.request as request
import zipfile
from project import logger

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.zip_file):
            filename, headers = request.urlretrieve(
            url = self.config.source_url,
            filename = self.config.zip_file)
            logger.info(f'{filename} downlod with following information: \n{headers}')
        else:
            logger.info(f"File already exixts of the size: {get_size(Path(self.config.zip_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_file
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.zip_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Error: {e}")

[2025-04-13 17:38:17,898: INFO: common: yaml file: yaml file\config.yaml loaded successfully]
[2025-04-13 17:38:17,920: INFO: common: yaml file: yaml file\params.yaml loaded successfully]
[2025-04-13 17:38:17,926: INFO: common: yaml file: yaml file\schema.yaml loaded successfully]
[2025-04-13 17:38:17,927: INFO: common: created directory at: artifacts]
[2025-04-13 17:38:17,929: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-13 17:38:50,832: INFO: 4149130257: artifacts/data_ingestion/data.zip downlod with following information: 
Connection: close
Content-Length: 792923
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "70351fa972cfd94c5f92762ef9b9e1b2c704730c94318be5bbdcceef3b0ecd1c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B04C:189510:196D4F:3E93E5:67FBA94B
Accep